In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import torch
import torchvision
from torchvision import transforms, datasets
from torch.utils.data.sampler import SubsetRandomSampler
import argparse
import torch.nn as nn
import torch.nn.functional as F
import gc
# torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 1, stride=1, padding=0)
        for j in range(3):
            for i in range(6):
                exec("self.Resconv1"+str(j)+str(i)+"="+"nn.Conv2d(32, 32, 3, stride=1, padding=1)")
                exec("self.Batch_norm1"+str(j)+str(i)+"="+"nn.BatchNorm2d(32, track_running_stats=False)")
                exec("self.Resconv2"+str(j)+str(i)+"="+"nn.Conv2d(32, 32, 3, stride=1, padding=1)")
                exec("self.Batch_norm2"+str(j)+str(i)+"="+"nn.BatchNorm2d(32, track_running_stats=False)")
#         self.Resconv1 = nn.Conv2d(32, 32, 3, stride=1, padding=1)
#         self.Batch_norm1 = nn.BatchNorm2d(32)
#         self.Resconv2 = nn.Conv2d(32, 32, 3, stride=1, padding=1)
#         self.Batch_norm2 = nn.BatchNorm2d(32)
        self.Transconv1 = nn.ConvTranspose2d(32, 32, 3, stride=2, padding=1, output_padding = 1)
        self.Transconv2 = nn.ConvTranspose2d(32, 32, 3, stride=2, padding=1, output_padding = 1)
        self.conv2 = nn.Conv2d(32, 3*256, 1, stride=1, padding=0)

        self.Maskconv1 = nn.Conv2d(3, 64, 7, stride=1, padding=3)
        for i in range(20):
            exec("self.Gateconv1"+str(i)+ "="+ "nn.Conv2d(64, 2*64, 5, stride=1, padding=2)")
            exec("self.Gateconv2"+str(i)+ "="+ "nn.Conv2d(2*64, 2*64, (1,1), stride=1, padding=0)")
            exec("self.Gateconv3"+str(i)+ "="+ "nn.Conv2d(64, 2*64, (1,5), stride=1, padding=(0,2))")
            exec("self.Gateconv4"+str(i)+ "="+ "nn.Conv2d(64, 64, (1,1), stride=1, padding=0)")
        self.Maskconv2 = nn.Conv2d(64, 1024, 1, stride=1, padding=0)
        self.Maskconv3 = nn.Conv2d(1024, 3*256, 1, stride=1, padding=0)

#         self.conv1.weight = nn.Parameter(self.weight_init(3, 32, (1,1), mask_type = None))
#         self.Resconv1.weight = nn.Parameter(self.weight_init(32, 32, (3,3), mask_type = None))
#         self.Transconv1.weight = nn.Parameter(self.weight_init(32, 32, (3,3), mask_type = None))
#         self.Resconv2.weight = nn.Parameter(self.weight_init(32, 32, (3,3), mask_type = None))
#         self.Transconv2.weight = nn.Parameter(self.weight_init(32, 32, (3,3), mask_type = None))
#         self.conv2.weight = nn.Parameter(self.weight_init(32, 3*256, (1,1), mask_type = None))

        self.Maskconv1.weight = nn.Parameter(self.weight_init(3, 64, (7,7), mask_type='A'))
        for i in range(20):
            exec("self.Gateconv1"+str(i)+".weight"+"="+"nn.Parameter(self.weight_init(64, 2*64, (5,5), mask_type='C'))")
            exec("self.Gateconv2"+str(i)+".weight"+"="+"nn.Parameter(self.weight_init(2*64, 2*64, (1,1), mask_type=None))")
            exec("self.Gateconv3"+str(i)+".weight"+"="+ "nn.Parameter(self.weight_init(64, 2*64, (1,5), mask_type='B'))")
            exec("self.Gateconv4"+str(i)+".weight"+"="+"nn.Parameter(self.weight_init(64, 64, (1,1), mask_type='B'))")
        self.Maskconv2.weight = nn.Parameter(self.weight_init(64, 1024, (1,1), mask_type='B'))
        self.Maskconv3.weight = nn.Parameter(self.weight_init(1024, 3*256, (1,1), mask_type='B'))

    def weight_init(self, in_channel, num_outputs, kernel_shape, mask_type=None):
        kernel_h, kernel_w = kernel_shape
        center_h = kernel_h // 2
        center_w = kernel_w // 2

        mask = np.zeros((num_outputs, in_channel, kernel_h, kernel_w), dtype=np.float32)
        if mask_type is not None:
            mask[:, :, :center_h, :] = 1
            if mask_type == 'A':
                mask[:, :, center_h, :center_w] = 1
            if mask_type == 'B':
                mask[:, :, center_h, :center_w+1] = 1
        else:
            mask[:, :, :, :] = 1

        weights_shape = [num_outputs, in_channel, kernel_h, kernel_w]
        weights = torch.empty(weights_shape, requires_grad=True)
        weights = nn.init.xavier_normal_(weights)
        weights = weights*torch.from_numpy(mask)
        return weights

    def prior_network(self, hr_images):
        conv1 = self.Maskconv1(hr_images)
        inputs = conv1
        state = conv1
        for i in range(20):
            inputs, state = self.gated_conv2d(inputs, state, [5, 5], i)
        conv2 = self.Maskconv2(inputs)
        conv2 = F.relu(conv2)
        prior_logits = self.Maskconv3(conv2)
        prior_logits = torch.cat((prior_logits[:, 0::3, :, :], prior_logits[:, 1::3, :, :], prior_logits[:, 2::3, :, :]), 1)
        return prior_logits

    def conditioning_network(self, lr_images):
        res_num = 6
        inputs = lr_images
        inputs = self.conv1(inputs)
        for i in range(2):
            for j in range(res_num):
                inputs = self.resnet_block(inputs, i, j)
            inputs = eval("self.Transconv"+str(i+1))(inputs)
            inputs = F.relu(inputs)
        for i in range(res_num):
            inputs = self.resnet_block(inputs, 2, i)
        conditioning_logits = self.conv2(inputs)
        return conditioning_logits

#     def batch_norm(self, x):
#         bn = nn.BatchNorm2d(x.shape[1], affine=True, track_running_stats=True)
#         return bn(x)

    def resnet_block(self, inputs, i, j):
        conv1 = eval("self.Resconv1"+str(i)+str(j))(inputs)
        bn1 = eval("self.Batch_norm1"+str(i)+str(j))(conv1)
        relu1 = F.relu(bn1)
        conv2 = eval("self.Resconv2"+str(i)+str(j))(relu1)
        bn2 = eval("self.Batch_norm2"+str(i)+str(j))(conv2)
        output = inputs + bn2
        return output


    def gated_conv2d(self, inputs, state, kernel_shape, i):
        batch_size, in_channel, height, width  = list(inputs.size())
        kernel_h, kernel_w = kernel_shape
        left = eval("self.Gateconv1"+str(i))(state)
        left1 = left[:, 0:in_channel, :, :]
        left2 = left[:, in_channel:, :, :]
        left1 = F.tanh(left1)
        left2 = F.sigmoid(left2)
        new_state = left1 * left2
        left2right = eval("self.Gateconv2"+str(i))(left)
        right = eval("self.Gateconv3"+str(i))(inputs)
        right = right + left2right
        right1 = right[:, 0:in_channel, :, :]
        right2 = right[:, in_channel:, :, :]
        right1 = F.tanh(right1)
        right2 = F.sigmoid(right2)
        up_right = right1 * right2
        up_right = eval("self.Gateconv4"+str(i))(up_right)
        outputs = inputs + up_right
        return outputs, new_state

    def forward(self, lr_images, hr_images):
        hr_images = hr_images - 0.5
        lr_images = lr_images - 0.5
        prior_logits = self.prior_network(hr_images)
        conditioning_logits = self.conditioning_network(lr_images)
        return prior_logits, conditioning_logits


In [4]:
def softmax_loss(logits, labels):
    logits = torch.reshape(logits, [-1, 256])
    labels = labels.to(torch.int64)
    labels = torch.reshape(labels, [-1])
    return F.cross_entropy(logits, labels)

def train(model, device, train_loader_lr, train_loader_hr, optimizer, epoch):
    model.train()
#     for name, param in model.named_parameters():
#                 if param.requires_grad:
#                     print(name, param.data)
    for (batch_idx, (data, t)), (target, p) in zip(enumerate(train_loader_lr), train_loader_hr):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        prior_logits, conditioning_logits = model(lr_images = data, hr_images = target)
        l1 = softmax_loss(prior_logits+conditioning_logits, torch.floor(target*255))
        l2 = softmax_loss(conditioning_logits, torch.floor(target*255))
        loss = l1+l2
        loss3 = softmax_loss(prior_logits, torch.floor(target*255))
        loss.backward()
        optimizer.step()
        if batch_idx % 1 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), (len(train_loader_lr)*len(data)),
                100. * batch_idx / len(train_loader_lr), loss.item()))

        if batch_idx % 100 == 0 and batch_idx != 0:
#             for name, param in model.named_parameters():
#                 if param.requires_grad:
#                     print(name, param.data)
            sample(model, data, target, len(data), mu=1.1, step=epoch*batch_idx)



def test(model, device, test_loader_lr, test_loader_hr):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for (data, t), (target, p) in zip(test_loader_lr, test_loader_hr):
            data, target = data.to(device), target.to(device)
            prior_logits, conditioning_logits = model(lr_images = data, hr_images = target)
            l1 = softmax_loss(prior_logits+conditioning_logits, torch.floor(target*255))
            l2 = softmax_loss(conditioning_logits, torch.floor(target*255))
            test_loss += l1+l2 # sum up batch loss

    test_loss /= len(test_loader_lr)*32
    print("test_loss : ", test_loss.item())

In [5]:
def logits_2_pixel_value(logits, mu=1.1):
    # print("convert")
#     print("logits", logits)
    rebalance_logits = logits * mu
    probs = softmax(rebalance_logits)
    pixel_dict = torch.arange(0, 256, dtype=torch.float32)
    pixels = torch.sum(probs*pixel_dict, dim=1)
    return pixels/255

def softmax(x):
    # print("cal")
    """Compute softmax values for each sets of scores in x."""
    # x = x.detach().numpy()
    a, b = torch.max(x, -1, keepdim=True, out=None)
    e_x = torch.exp(x - a)
    return e_x / e_x.sum(dim=-1, keepdim =True) # only difference

def sample(model, data, target, batch_size, mu=1.1, step=None):
    with torch.no_grad():
        np_lr_imgs = data
        np_hr_imgs = target
        c_logits = model.conditioning_network
        p_logits = model.prior_network
        gen_hr_imgs = torch.zeros((batch_size, 3, 32, 32), dtype=torch.float32)
        np_c_logits = c_logits(torch.floor(np_lr_imgs*255))
        print("torch.floor(np_lr_imgs*255)", torch.floor(np_lr_imgs*255)[0])
        print("c_logits", np_c_logits[0].shape, np_c_logits[0])
        print("p_logits", p_logits(gen_hr_imgs)[0].shape, p_logits(gen_hr_imgs)[0])
        print("target", torch.floor(target*255)[0])
        for i in range(32):
            print(i)
            for j in range(32):
                for c in range(3):
#                     print(i,j,c)
                    np_p_logits = p_logits(gen_hr_imgs)
                    new_pixel = logits_2_pixel_value(np_c_logits[:, c*256:(c+1)*256, i, j] + np_p_logits[:, c*256:(c+1)*256, i, j], mu=mu)
                    gen_hr_imgs[:, c, i, j] = new_pixel
                    # try:
                    # 	for obj in gc.get_objects():
                    # 		if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
                    # 			print(type(obj), obj.size())
                    # except:
                    # 	print("error")
        samples_dir = "/home/eee/ug/15084005/DIH/samples/"
        print("sample")
        save_samples(np_lr_imgs, samples_dir + '/lr_' + str(mu*10) + '_' + str(step))
        save_samples(np_hr_imgs, samples_dir + '/hr_' + str(mu*10) + '_' + str(step))
        save_samples(gen_hr_imgs, samples_dir + '/generate_' + str(mu*10) + '_' + str(step))

def save_samples(np_imgs, img_path):
    print("save")
    torchvision.utils.save_image(np_imgs[0, :, :, :], img_path+".jpg")


In [6]:
def main():
#     parser = argparse.ArgumentParser()
#     parser.add_argument("--use_gpu", type = bool, default = True, help = "use or not gpu")
#     parser.add_argument("--num_epoch", type = int, default = 30, help = "no of epoch")
#     parser.add_argument("--batch_size", type = int, default = 32, help = "batch size")
#     parser.add_argument("--learning_rate", type = float, default = 4e-4, help = "learning rate")
#     args = parser.parse_args()
    use_gpu = False
    num_epoch = 30
    batch_size = 32
    learning_rate = 4e-4

    use_cuda = use_gpu and torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
#     torch.cuda.set_device(0)

    data = datasets.ImageFolder(root = '/home/eee/ug/15084005/DIH/CelebA/CelebA/train/', transform = transforms.Compose([transforms.Resize((8,8), interpolation=2), transforms.ToTensor()]))
    target = datasets.ImageFolder(root = '/home/eee/ug/15084005/DIH/CelebA/CelebA/train/', transform = transforms.Compose([transforms.Resize((32,32), interpolation=2), transforms.ToTensor()]))

    data_test = datasets.ImageFolder(root = '/home/eee/ug/15084005/DIH/CelebA/CelebA/test/', transform = transforms.Compose([transforms.Resize((8,8), interpolation=2), transforms.ToTensor()]))
    target_test = datasets.ImageFolder(root = '/home/eee/ug/15084005/DIH/CelebA/CelebA/test/', transform = transforms.Compose([transforms.Resize((32,32), interpolation=2), transforms.ToTensor()]))


    #test_split = .05
    #shuffle_dataset = True
    #random_seed= 42
    # Creating data indices for training and validation splits:
    # dataset_size = len(image_data_hr)
    # indices = list(range(dataset_size))
    # split = int(np.floor(test_split * dataset_size))
    # if shuffle_dataset :
    # 	np.random.seed(random_seed)
    # 	np.random.shuffle(indices)
    # train_indices, test_indices = indices[split:], indices[:split]

    # data = [image_data_lr[x] for x in train_indices]
    # target = [image_data_hr[x] for x in train_indices]

    # data_test = [image_data_lr[x] for x in train_indices]
    # target_test = [image_data_hr[x] for x in train_indices]

    train_sampler_lr = torch.utils.data.sampler.SequentialSampler(data)
    train_sampler_hr = torch.utils.data.sampler.SequentialSampler(target)

    test_sampler_lr = torch.utils.data.sampler.SequentialSampler(data_test)
    test_sampler_hr = torch.utils.data.sampler.SequentialSampler(target_test)

    train_loader_lr = torch.utils.data.DataLoader(data, batch_size=batch_size, sampler=train_sampler_lr)
    train_loader_hr = torch.utils.data.DataLoader(target, batch_size=batch_size, sampler=train_sampler_hr)

    test_loader_lr = torch.utils.data.DataLoader(data_test, batch_size=batch_size, sampler=test_sampler_lr)
    test_loader_hr = torch.utils.data.DataLoader(target_test, batch_size=batch_size, sampler=test_sampler_hr)

    model = Net()
    model = model.to(device)

    optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.95)
    print(sum(p.numel() for p in model.parameters() if p.requires_grad))
#     for parameter in model.parameters():
#         print(parameter)
#     print(list(model.Maskconv1.parameters()))
#     print(list(model.Gateconv4.parameters()))
    for epoch in range(1, num_epoch+1):
        train(model, device, train_loader_lr, train_loader_hr, optimizer, epoch)
        # sample(model, train_loader_lr, train_loader_hr, mu=1.1, step=epoch)
        test(model, device, test_loader_lr, test_loader_hr)

In [7]:
main()

6576192
Train Epoch: 1 [0/192608 (0%)]	Loss: 11.732662
Train Epoch: 1 [32/192608 (0%)]	Loss: 11.636530
Train Epoch: 1 [64/192608 (0%)]	Loss: 11.418681
Train Epoch: 1 [96/192608 (0%)]	Loss: 11.248533
Train Epoch: 1 [128/192608 (0%)]	Loss: 11.146524
Train Epoch: 1 [160/192608 (0%)]	Loss: 11.119799
Train Epoch: 1 [192/192608 (0%)]	Loss: 11.122786
Train Epoch: 1 [224/192608 (0%)]	Loss: 11.146030
Train Epoch: 1 [256/192608 (0%)]	Loss: 11.114670
Train Epoch: 1 [288/192608 (0%)]	Loss: 11.110754
Train Epoch: 1 [320/192608 (0%)]	Loss: 11.105581
Train Epoch: 1 [352/192608 (0%)]	Loss: 11.100151
Train Epoch: 1 [384/192608 (0%)]	Loss: 11.099194
Train Epoch: 1 [416/192608 (0%)]	Loss: 11.097786
Train Epoch: 1 [448/192608 (0%)]	Loss: 11.097876
Train Epoch: 1 [480/192608 (0%)]	Loss: 11.097520
Train Epoch: 1 [512/192608 (0%)]	Loss: 11.095774
Train Epoch: 1 [544/192608 (0%)]	Loss: 11.096651
Train Epoch: 1 [576/192608 (0%)]	Loss: 11.096519
Train Epoch: 1 [608/192608 (0%)]	Loss: 11.095449
Train Epoch: 1 [6

KeyboardInterrupt: 